In [1]:
import pandas as pd
import json

users = pd.read_csv('relax_challenge/takehome_users.csv',  encoding='cp1252', engine='python')
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [2]:
users['invited_by_user_id'] = users['invited_by_user_id'].fillna(0).apply(int).apply(str).replace('0','')
users['object_id'] = users['object_id'].apply(str)
users['creation_time'] = pd.to_datetime(users['creation_time'])
users['creation_source'] = users['creation_source'].astype('category')
users['opted_in_to_mailing_list'] = users['opted_in_to_mailing_list'].astype('category')
users['enabled_for_marketing_drip'] = users['enabled_for_marketing_drip'].astype('category')
users['org_id'] = users['org_id'].astype('category')
users['last_session_creation_time'] = pd.to_datetime(users['last_session_creation_time'],unit='s') # unix timestamp
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240


In [3]:
len(users)

12000

<B>Defining an "adopted user" as a user who has logged into the product on three separate days in at least one seven-day period, identify which factors predict future user adoption.

We suggest spending 1-2 hours on this, but you're welcome to spend more or less. Please send us a brief writeup of your findings (the more concise, the better -- no more than one page), along with any summary tables, graphs, code, or queries that can help us understand your approach. Please note any factors you considered or investigation you did, even if they did not pan out. Feel free to identify any further research or data you think would be valuable.</B>

In [4]:
engagement = pd.read_csv('relax_challenge/takehome_user_engagement.csv',  encoding='cp1252', engine='python')
engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [5]:
engagement['time_stamp'] = pd.to_datetime(engagement['time_stamp'])
engagement['user_id'] = engagement['user_id'].apply(str)
engagement['visited'] = engagement['visited'].apply(str)
engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [6]:
sorted =engagement.sort_values(by=['time_stamp','user_id'])
sorted.loc[(sorted.user_id == '20'),]

,time_stamp,user_id,visited
310,2014-03-11 11:46:38,20,1
311,2014-03-12 11:46:38,20,1
312,2014-03-13 11:46:38,20,1
313,2014-05-07 11:46:38,20,1
314,2014-05-11 11:46:38,20,1
315,2014-05-28 11:46:38,20,1
316,2014-05-29 11:46:38,20,1


In [7]:
user = sorted['user_id'].unique()
print(len(user), sorted['visited'].max(), sorted['visited'].min())

8823 1 1


In [8]:
%time

#counter = 0
adopted = set()
for i in user:
    if users.loc[(users.object_id == i),] is None:
        print( str(i) + ' not in user list')
        continue
    visited = sorted.loc[(sorted.user_id == i),]
    #print('user '+ i  + "; total visited " + str(len(visited)))
    for j in visited['time_stamp']:
        k = j + pd.DateOffset(days=7)
        period = visited.loc[(visited.time_stamp >= j) & (visited.time_stamp <= k),]
        #print('user ' + i + '; start '+str(j) + "  " + '; end ' + str(k) + "; visited " +str(len(period)))
        #if len(period) >= 3:
        #    print('   visited ' + str(len(period)))
        unique = len(period['time_stamp'].dt.date.unique())
        if unique != len(period):
            print('same day visit for user ' +i)
        if  unique >= 3:
            adopted.add(i)
            break
            #print('   adopted ' + str(unique))
        
    #counter +=1
    #if counter > 2000:
        #break

print(len(adopted))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs
1656


In [9]:
print(adopted)

{'5965', '509', '7431', '2681', '10958', '11642', '2199', '7882', '11924', '10860', '3757', '5160', '4467', '4248', '10937', '11195', '9434', '3025', '2481', '3194', '9623', '6609', '3824', '4181', '5479', '4580', '2672', '9948', '8048', '7386', '6050', '5101', '6266', '10750', '5528', '5156', '3240', '5297', '1464', '9768', '2739', '3239', '8673', '133', '2771', '7518', '5725', '11762', '1821', '1233', '5828', '6992', '11186', '3749', '7720', '5407', '4837', '10683', '506', '7172', '9985', '6427', '9722', '8442', '8837', '2615', '10753', '1999', '3378', '418', '3217', '10911', '529', '1770', '572', '4935', '5892', '347', '9727', '4844', '9691', '10580', '8466', '5415', '2011', '9303', '4093', '1156', '1681', '11913', '1506', '9593', '2587', '8806', '1709', '11412', '10222', '5432', '11964', '1783', '3577', '7896', '9079', '11176', '1825', '8408', '3525', '10159', '9222', '10746', '3804', '10412', '8955', '11965', '2073', '9754', '10520', '8111', '7153', '7004', '10363', '11388', '7951

In [10]:
len(adopted) / len(user) 

0.18769126147568854

In [11]:
'20' in adopted

True

<font color='red'>

## The above fund around 18% of users is the "adopted user". 

## The first thought was to use the rolling window. However, as the timestamp is not a daily record, the specification for the rolling window does not fit the requirement.  

## Therefore, after sorting the data according to the logged time, a dual loop was executed to find the user who is qualified.  

## The complexity is n^2, which is not the best algorithm but does perform the job.

</font>  